# Proyecto Final: Procesamiento de Lenguaje Natural

## Definición del proyecto

El objetivo de este proyecto es utilizar una Red Neuronal Recurrente usando LSTM (Long-Short Term Memory) para hacer traducción de Inglés a Italiano. En los datos de entrenamiento que se nos proporcionan tenemos un par de frases; la frase en inglés y su correspondiente traducción a italiano. 

Para lograr esto, utilizaremos una red "Sequence to Sequence" en el que tendremos dos redes neuronales recurrentes para transformar una secuencia en otra; la primera nos servira como un "Encoder" para transofrmar una entrada de text a un vector mientras que la segunda nos servira para poder

![Red Sequence-Sequence](./imagenes/encoder-decoder.png)

## Requerimientos

Para poder realizar este proyecto decidimos usar PyTorch para poder utilizar tensores y aprovechar el GPU de la computadora para el procesamiento (algo que con numpy no se puede hacer). La versión de Pytorch que utilizamos para este proyecto tiene las siguientes caracteristicas:
 - Version 1.1
 - Windows 10
 - Cuda 10.1
 - Python 3.7
 - Conda

In [ ]:
#conda install pytorch torchvision cudatoolkit=10.0 -c pytorch

## Imports

Importamos todas la bibliotecas que vamos a estar utilizando. En este proeycto utilizaremos algunas librerias generales como lo son `string` y `re` que son de uso común durante cualquier proyecto de procesamiento de texto.

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

# Cargamos torch
import torch
# Cargamos la biblioteca de redes neuronales de pytorch
import torch.nn as nn
# Biblioteca de optimización de algoritmos
from torch import optim
# Funciones para entrenar la red neuronal.
import torch.nn.functional as F

# Buscamos si el dispositivo donde se ejecuita el algoritmo tiene cuda instalado
# en caso de que no lo tenga usará el CPU para el entrenamiento.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Carga de texto
En este proyecto el proceso de carga de texto se realiza utilizando pares de frases la primera en inglés seguido de un tab y despues en italiano.

```
we therefore respect whatever parliament may decide 	quindi noi rispettiamo le eventuali decisioni in materia del parlamento

```

Como vimos en clase la primera etapa será construir nuestros vectores one hot de las palabras que contiene el corpus de entrenamiento. Para esto generaremos un indice unico para cada una de las palabras el cual utilizará para ubicarse dentro del vector. 

![One Hot](./imagenes/onehot.png)

Lo primero que haremos sera crear todas las funciones para limpiar los datos. Primero, pasamos de Unicode a ASCII y después normalizamos el texto quitando los simbolos especiales.

In [10]:
def unicodeToAscii(w):
    """Transforma la palabra de entrada de Unicode a Ascii 
    basado en https://stackoverflow.com/a/518232/2809427"""
    
    return ''.join(
        c for c in unicodedata.normalize('NFD', w)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeStr(s):
    """Elimina signos de puntuación y transforma el string a minúsculas"""
    
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

Desarrollamos una clase llamada Lenguaje, el cual contendrá el las funciones necesarias para pasar de la palabra al indíce y del indice de regreso a la palabra utilizando diccionarios. 

In [4]:
#Defininimos lot tokens de inciio de oracion y de fin de oracion
BOS = 0
EOS = 1

class Language:
    """
    Esta clase sirve para agregar las palabras al diccionario y asignarles un 
    indice. Cuenta con las funciones necesarias tanto para pasar de un indice
    a una palabra como de una palabara a un indice.
    """
    def __init__(self, name):
        """Inicializa las variables locales de la clase que son los diccionarios en los 
        que se almacena la informacion de las palabras"""
        self.name = name
        self.word2index={} # Nos servirá para convertir de una palabra a un índice
        self.word2count={} # Nos servira para contar la frecuencia en que aparece una palabra
        self.index2word={} # Nos servirá para pasar de un índice a una palabra
        self.total = 2     # Noss servirá para contar el total de palabras únicas
        
    def procSentences(self, sentence):
        """Procesa una oración para ser agregada a los diccionarios del lenguaje"""
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        """Agrega las palabras a los diccionarios y actualiza los contadores"""
        if word not in self.word2index:
            self.word2index[word] = self.total
            self.word2count[word] = 1 
            self.index2word[self.total] = word
            self.total += 1
        
        else:
            self.word2count[word] += 1  
        

Hacemos la función que nos permitirá leer las frases que vienen en el archivo de entrenamiento y las almacene.

In [44]:
def readFile(lang1, lang2, reverse=False):
    """Lee el archivo de entrenamiento del traductor, lo limpia y asigna una clase Language
    para almacenar los datos del lenguaje. Se permite revertir el orden para realizar pruebas
    traduciendo de manera inversa el lenguaje.
    """
    
    print("Loading Files...")

    # Abrimos el archivo y lo dividmos por salto de linea para obtener
    # los pares en una sola linea.
    lines = open('./corpus/data3.test', encoding='utf-8').read().strip().split('\n')
    #print("Found {} lines".format(len(lines)))

    # Dividimos los pares en una linea en su lenguaje de entrada y lenguaje de salida
    pairs = [[normalizeStr(s) for s in l.split('\t')] for l in lines]

    # Instanceamos las clases Lang
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Language(lang2)
        output_lang = Language(lang1)
    else:
        input_lang = Language(lang1)
        output_lang = Language(lang2)

    return input_lang, output_lang, pairs

Cargamos los datos del set de entrenamieto a dos clases de tipo Language: input(inglés) y output(italiano). 

In [47]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readFile(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.procSentences(pair[0])
        output_lang.procSentences(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.total)
    print(output_lang.name, output_lang.total)
    return input_lang, output_lang, pairs


output_lang, input_lang, pairs = prepareData('eng', 'ita', True)
print(random.choice(pairs))

Loading Files...
Read 900 sentence pairs
Counting words...
Counted words:
ita 4420
eng 3291
['la situazione e grave al punto che oggi anche nell unione europea e evidente il nesso fra disoccupazione e poverta come si desume dal fatto molto preoccupante che la disoccupazione raggiunge in media il percento nelle regioni piu colpite da questo problema che coincidono con le zone povere mentre nelle regioni che corrispondono alle zone ricche la disoccupazione e di appena il percento', 'we have a serious situation in which in the european union today there is a genuine link between unemployment and poverty as demonstrated by the very worrying fact that unemployment has reached on average in the regions worst affected regions which also happen to be poor areas whilst in the regions with the lowest unemployment corresponding to the richer areas unemployment stands at just ']


## Red Neuronal Recurrente (RNR)
Una red neuronal recurrente es un tipo de red que utiliza secuancias para preoducir una salida tomando en cuenta la organización de la entrada de la red. Uno de los principales retos con las RNR es que el modelo de la red normalmente produce como máximo una salida por cada dato de entrada y en el ejemplo de traducción tenemos palabras que pueden producir más de una salida por lo que el modelo tradicional de RNR no nos es completamente útil para traducción.

### Seq2Seq (Encoder-Decoder)
Para solucionar esto, utilizaremos dos RNR una que nos servirá de encoder y otra que nos servirá de decoder de la traducción. El objetivo con esto es que sin importar la longitud de la frase de entrada o su organización, en cuanto a la posición de las palabras en el texto, podamos hacer la traducción de la manera más precisa posible. Esto se logra generando un vector intermedio en el que se intenta almacenar el significado de la frase de entrada a través del encoder y despues este significado es interpretado por el decoder en la salida.

![encoder-decoder](./imagenes/paper-encode.png)

Al final, viendo la imagen #1 del Notebook, no importa si la entrada es `le chat es noir` o `le chat noir` como el significado es el mismo debería producir el mismo resultado.

### Encoder

Para el encoder usaremos una unidad GRU (Unidad Recurrente multicapa con compuertas), la cual, para cada una de las entradas, hará el siguiente cálculo.

\begin{array}{ll}
            r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
            z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
            n_t = \tanh(W_{in} x_t + b_{in} + r_t * (W_{hn} h_{(t-1)}+ b_{hn})) \\
            h_t = (1 - z_t) * n_t + z_t * h_{(t-1)}
\end{array}

Donde:
\begin{array}{ll}
     h_{(t)} -> \text{Estado oculto en el tiempo t}\\
     x_t -> \text{La entrada en el tiempo t} \\
     h_{(t-1)} -> \text{Estado oculto en el tiempo t-1 o el inicial para t=0} \\
     z_t -> \text{Gate de actualización} \\
     r_t -> \text{Gate de reset} \\
     n_t -> \text{}\\
     \sigma -> \text{Funcion sigmoide} \\
     * -> \text{Producto de Hadamard} \\
\end{array}
        

## Referencias
 
 - Basile, Pierpaolo, et al. “Bi-Directional LSTM-CNNs-CRF for Italian Sequence Labeling.” Proceedings of the Fourth Italian Conference on Computational Linguistics CLiC-It 2017, 2017, pp. 18–23., doi:10.4000/books.aaccademia.2339.
 - Raval, Siraj. “Recurrent Neural Network - The Math of Intelligence (Week 5).” YouTube, YouTube, 19 July 2017, www.youtube.com/watch?v=BwmddtPFWtA.
 - Raval, Siraj. “LSTM Networks - The Math of Intelligence (Week 8).” YouTube, YouTube, 9 Aug. 2017, www.youtube.com/watch?v=9zhrxE5PQgY. 
 - Trask, Andrew. “Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN).” Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN) - i Am Trask, iamtrask.github.io/2015/11/15/anyone-can-code-lstm/.
 - PyTorch. “Translation with a Sequence to Sequence Network and Attention.” Translation with a Sequence to Sequence Network and Attention - PyTorch Tutorials 1.1.0 Documentation, pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html.
 - Ilya Sutskever. “Sequence to Sequence Learning with Neural Networks.” Cornell University, 10 Sept. 2014, arxiv.org/abs/1409.3215v3.
 - Cho, Kyunghyun, et al. “Learning Phrase Representations Using RNN Encoder–Decoder for Statistical Machine Translation.” Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP), 3 Sept. 2014, doi:10.3115/v1/d14-1179.


